In [265]:
import requests
import functools
import re
from itertools import chain
from bs4 import BeautifulSoup
from urllib.parse import quote_plus

In [271]:
def get(url, query):
    quoted_query = quote_plus(query)
    underscored_query = query.replace(' ', '_')
    full_url = '{}?q={}'.format(url, quoted_query)
    r = requests.get(full_url)
    status = r.status_code
    if status is 200:
        soup = BeautifulSoup(r.text, 'html.parser')
        return soup
    else:
        print('Error code: {}; URL: {}'.format(status, full_url))

In [280]:
def yahoo(query): return yahooParser(get('https://search.yahoo.com/search;', query))
def yahooParser(soup):
    matchURLsThatArentYahoosIP = re.compile("(?=^http.)(?!^http:\/\/98)")
    selectResultsTag           = lambda soup : soup.select("ol.searchCenterMiddle")[0]
    removeNewsAndSpecialLinks  = lambda soup : soup.select("div.wrapstar")+soup.select("div.algo")
    selectSearchResults        = lambda soup : removeNewsAndSpecialLinks(selectResultsTag(soup))
    removeInvalidURLs          = lambda url  : matchURLsThatArentYahoosIP.search(url)
    selectLinkTag              = lambda div  : div.find_all('a', href=True)
    selectLinkURL              = lambda link : link["href"]
    flatmap                    = lambda fun, lst : list(chain.from_iterable(map(fun,lst)))
    
    return list(filter(removeInvalidURLs,
                map(selectLinkURL,
                flatmap(selectLinkTag,
                        selectSearchResults(soup)))))

In [297]:
def google(query): return googleParser(get('https://www.google.com/search', query))
def googleParser(soup):
    def filter_1(tag) : return tag.name == 'div' and tag.parent.name == 'ol' and tag['class'] == ['g']
    def filter_2(tag): return tag.name == 'a' and tag.parent.name == 'h3' and tag.text != 'News for {}'.format(query)
    def strip_href(href): return href.strip('/url?q=').split('&')[0]
    
    return [strip_href(tag.h3.a['href'])
           for tag in soup.find_all(filter_1)
           if tag.find_all(filter_2)]

In [298]:
def bing(query): return bingParser(get('http://www.bing.com/search', query))
def bingParser(soup): 
    return [t.h2.a['href'] for t in soup.find_all('li', 'b_algo')]

In [299]:
yahoo("test")

['http://www.speedtest.net/',
 'http://www.thefreedictionary.com/test',
 'http://www.test.com/',
 'https://www.speakeasy.net/speedtest/',
 'http://www.dictionary.com/browse/test',
 'https://en.wikipedia.org/wiki/Test_(student_assessment)',
 'http://www.merriam-webster.com/dictionary/test',
 'https://en.wikipedia.org/wiki/Test',
 'http://www.tests.com/',
 'http://www.queendom.com/tests/index.htm']

In [300]:
google("test")

['https://www.test.com/',
 'http://www.speedtest.net/',
 'https://en.wikipedia.org/wiki/Test',
 'https://www.speakeasy.net/speedtest/',
 'http://speedtest.xfinity.com/',
 'https://www.16personalities.com/free-personality-test',
 'https://www.google.com/webmasters/tools/mobile-friendly/',
 'http://www.springer.com/statistics/journal/11749',
 'http://www.att.com/speedtest/',
 'https://implicit.harvard.edu/implicit/takeatest.html']

In [301]:
bing("test")

['http://www.speedtest.net/',
 'http://www.test.com/',
 'http://www.merriam-webster.com/dictionary/test',
 'http://www.thefreedictionary.com/test',
 'http://www.dictionary.com/browse/test',
 'https://en.wikipedia.org/wiki/Test_(student_assessment)',
 'https://www.speakeasy.net/speedtest/',
 'https://en.wikipedia.org/wiki/Test',
 'http://www.tests.com/']